In [45]:
from lightgbm import LGBMRegressor
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, RidgeCV
from sklearn.svm import LinearSVR

In [46]:
df = pd.read_csv('../input/renfe.csv', index_col=0)

print(df.shape)
df.head()

(2579771, 9)


,insert_date,origin,destination,start_date,end_date,train_type,price,train_class,fare
0,2019-04-19 05:31:43,MADRID,SEVILLA,2019-05-29 06:20:00,2019-05-29 09:16:00,AV City,38.55,Turista,Promo
1,2019-04-19 05:31:43,MADRID,SEVILLA,2019-05-29 07:00:00,2019-05-29 09:32:00,AVE,53.40,Turista,Promo
2,2019-04-19 05:31:43,MADRID,SEVILLA,2019-05-29 07:30:00,2019-05-29 09:51:00,AVE,47.30,Turista,Promo
3,2019-04-19 05:31:43,MADRID,SEVILLA,2019-05-29 08:00:00,2019-05-29 10:32:00,AVE,69.40,Preferente,Promo
4,2019-04-19 05:31:43,MADRID,SEVILLA,2019-05-29 08:30:00,2019-05-29 11:14:00,ALVIA,NaN,Turista,Promo


In [47]:
for col in ['insert_date']:
    date_col = pd.to_datetime(df[col])
    df[col + '_hour'] = date_col.dt.hour
    df[col + '_minute'] = date_col.dt.minute
    df[col + '_second'] = date_col.dt.second
    df[col + '_weekday'] = date_col.dt.day_name
    df[col + '_day'] = date_col.dt.day
    df[col + '_month'] = date_col.dt.month
    df[col + '_year'] = date_col.dt.year

    del df[col]

In [56]:
for col in ['start_date']:
    date_col = pd.to_datetime(df[col])
    df[col + '_hour'] = date_col.dt.hour
    df[col + '_minute'] = date_col.dt.minute
    df[col + '_second'] = date_col.dt.second
    df[col + '_weekday'] = date_col.dt.day_name
    df[col + '_day'] = date_col.dt.day
    df[col + '_month'] = date_col.dt.month
    df[col + '_year'] = date_col.dt.year

    del df[col]

In [57]:
for col in ['end_date']:
    date_col = pd.to_datetime(df[col])
    df[col + '_hour'] = date_col.dt.hour
    df[col + '_minute'] = date_col.dt.minute
    df[col + '_second'] = date_col.dt.second
    df[col + '_weekday'] = date_col.dt.day_name
    df[col + '_day'] = date_col.dt.day
    df[col + '_month'] = date_col.dt.month
    df[col + '_year'] = date_col.dt.year

    del df[col]

In [ ]:
df.head()

In [49]:
df.isnull().sum()

origin                      0
destination                 0
start_date                  0
end_date                    0
train_type                  0
price                  310681
train_class              9664
fare                     9664
insert_date_hour            0
insert_date_minute          0
insert_date_second          0
insert_date_weekday         0
insert_date_day             0
insert_date_month           0
insert_date_year            0
dtype: int64

In [50]:
df.dropna(inplace=True)

In [51]:
for col in df.columns:
    print(col, ":", df[col].unique().shape[0])

origin : 5
destination : 5
start_date : 7729
end_date : 10076
train_type : 15
price : 225
train_class : 6
fare : 7
insert_date_hour : 24
insert_date_minute : 60
insert_date_second : 60
insert_date_weekday : 1
insert_date_day : 29
insert_date_month : 2
insert_date_year : 1


We see that all the data is categorical in this case. We can one-hot-encode them afterwards.

Also, it seems like there is only one year in the dataset. We can safely drop that column.

In [52]:
columns_to_drop = [col for col in df.columns if df[col].unique().shape[0] == 1]
df.drop(columns=columns_to_drop, inplace=True)

In [53]:
df.head()

,origin,destination,start_date,end_date,train_type,price,train_class,fare,insert_date_hour,insert_date_minute,insert_date_second,insert_date_day,insert_date_month
0,MADRID,SEVILLA,2019-05-29 06:20:00,2019-05-29 09:16:00,AV City,38.55,Turista,Promo,5,31,43,19,4
1,MADRID,SEVILLA,2019-05-29 07:00:00,2019-05-29 09:32:00,AVE,53.40,Turista,Promo,5,31,43,19,4
2,MADRID,SEVILLA,2019-05-29 07:30:00,2019-05-29 09:51:00,AVE,47.30,Turista,Promo,5,31,43,19,4
3,MADRID,SEVILLA,2019-05-29 08:00:00,2019-05-29 10:32:00,AVE,69.40,Preferente,Promo,5,31,43,19,4
5,MADRID,SEVILLA,2019-05-29 09:00:00,2019-05-29 11:38:00,AVE,60.30,Turista,Promo,5,31,43,19,4


In [54]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

<ipython-input-54-935c2e534e54>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr = df.corr()


,price,insert_date_hour,insert_date_minute,insert_date_second,insert_date_day,insert_date_month
price,1.000000,-0.000234,-0.020607,0.003265,-0.023150,0.025321
insert_date_hour,-0.000234,1.000000,-0.012808,-0.000976,-0.021370,-0.000949
insert_date_minute,-0.020607,-0.012808,1.000000,-0.024427,0.011191,0.044788
insert_date_second,0.003265,-0.000976,-0.024427,1.000000,-0.002255,0.003396
insert_date_day,-0.023150,-0.021370,0.011191,-0.002255,1.000000,-0.803938
insert_date_month,0.025321,-0.000949,0.044788,0.003396,-0.803938,1.000000


The only highly correlated feature we can observe is the between the start and end date (both day and month). We can drop off one of each.

In [58]:
df.drop(columns=['end_date_day', 'end_date_month'], inplace=True)

In [59]:
price_freq = df['price'].value_counts()
price_freq.head()

76.30    166085
28.35    141822
85.10    124541
60.30     83445
75.40     80246
Name: price, dtype: int64

In [60]:
price_freq.tail()

40.93    1
85.15    1
16.75    1
26.65    1
68.97    1
Name: price, dtype: int64

In [61]:
X_df = df.drop(columns='price')
y = df['price'].values

In [62]:
encoder = OneHotEncoder()
X = encoder.fit_transform(X_df.values)
X

<2269090x367 sparse matrix of type '<class 'numpy.float64'>'
	with 49919980 stored elements in Compressed Sparse Row format>

In [63]:
for category in encoder.categories_:
    print(category[:5])

['BARCELONA' 'MADRID' 'PONFERRADA' 'SEVILLA' 'VALENCIA']
['BARCELONA' 'MADRID' 'PONFERRADA' 'SEVILLA' 'VALENCIA']
['ALVIA' 'AV City' 'AVE' 'AVE-LD' 'AVE-MD']
['Cama G. Clase' 'Cama Turista' 'Preferente' 'Turista' 'Turista Plus']
['Adulto ida' 'Flexible' 'Grupos Ida' 'Individual-Flexible' 'Mesa']
[0 1 2 3 4]
[0 1 2 3 4]
[0 1 2 3 4]
[1 2 3 4 5]
[4 5]
[2 5 6 7 8]
[0 3 5 8 10]
[0]
[<bound method PandasDelegate._add_delegate_accessors.<locals>._create_delegator_method.<locals>.f of <pandas.core.indexes.accessors.DatetimeProperties object at 0x7d5eb5c83e80>>]
[1 2 3 4 5]
[4 5 6 7]
[2019]
[0 4 8 9 10]
[0 1 2 3 4]
[0]
[<bound method PandasDelegate._add_delegate_accessors.<locals>._create_delegator_method.<locals>.f of <pandas.core.indexes.accessors.DatetimeProperties object at 0x7d5eb5c839a0>>]
[2019]


In [64]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=2019
)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(2042181, 367)
(226909, 367)
(2042181,)
(226909,)


In [65]:
%%time
model = LinearRegression()
model.fit(X_train, y_train)

CPU times: user 1min 22s, sys: 1min 15s, total: 2min 37s
Wall time: 1min 39s


LinearRegression()

In [66]:
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

print("Train Score:", train_score)
print("Test Score:", test_score)

Train Score: 0.8639659531210708
Test Score: 0.8645044205218928


In [67]:
def compute_mse(model, X, y_true, name):
    y_pred = model.predict(X)
    mse = mean_squared_error(y_true, y_pred)
    print(f'Mean Squared Error for {name}: {mse}')

compute_mse(model, X_train, y_train, 'training set')
compute_mse(model, X_test, y_test, 'test set')

Mean Squared Error for training set: 90.46169003065548
Mean Squared Error for test set: 89.80471424657136


The MSE is pretty high, considering that the mean is only:

In [68]:
y_train.mean()

63.38648255468053

This teaches us to not trust a single evaluation metric! Therefore, there is still some room for improvement.

Instead of repeating ourselves, we will build a simple function called `evaluate`, which will print the score and MSE of our models on both the training and test sets.

In [69]:
def build_evaluate_fn(X_train, y_train, X_test, y_test):
    def evaluate(model):
        train_score = model.score(X_train, y_train)
        test_score = model.score(X_test, y_test)

        print("Train Score:", train_score)
        print("Test Score:", test_score)
        print()

        compute_mse(model, X_train, y_train, 'training set')
        compute_mse(model, X_test, y_test, 'test set')

    return evaluate

evaluate = build_evaluate_fn(X_train, y_train, X_test, y_test)

In [70]:
%%time
svm = LinearSVR()
svm.fit(X_train, y_train);

CPU times: user 3min 43s, sys: 558 ms, total: 3min 43s
Wall time: 3min 45s


LinearSVR()

In [71]:
evaluate(svm)

Train Score: 0.8447945180605373
Test Score: 0.8451163721258317

Mean Squared Error for training set: 103.21056030011344
Mean Squared Error for test set: 102.65486148173109


In [72]:
%%time
gbr = LGBMRegressor(n_estimators=1000)
gbr.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.593520 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 722
[LightGBM] [Info] Number of data points in the train set: 2042181, number of used features: 361
[LightGBM] [Info] Start training from score 63.386483
CPU times: user 2min 2s, sys: 0 ns, total: 2min 2s
Wall time: 2min 2s


LGBMRegressor(n_estimators=1000)

In [73]:
evaluate(gbr)

Train Score: 0.9703796155081951
Test Score: 0.9701595573248275

Mean Squared Error for training set: 19.697348582677012
Mean Squared Error for test set: 19.777858715073783


In [74]:
df.head()

,origin,destination,train_type,price,train_class,fare,insert_date_hour,insert_date_minute,insert_date_second,insert_date_day,...,start_date_second,start_date_weekday,start_date_day,start_date_month,start_date_year,end_date_hour,end_date_minute,end_date_second,end_date_weekday,end_date_year
0,MADRID,SEVILLA,AV City,38.55,Turista,Promo,5,31,43,19,...,0,<bound method PandasDelegate._add_delegate_acc...,29,5,2019,9,16,0,<bound method PandasDelegate._add_delegate_acc...,2019
1,MADRID,SEVILLA,AVE,53.40,Turista,Promo,5,31,43,19,...,0,<bound method PandasDelegate._add_delegate_acc...,29,5,2019,9,32,0,<bound method PandasDelegate._add_delegate_acc...,2019
2,MADRID,SEVILLA,AVE,47.30,Turista,Promo,5,31,43,19,...,0,<bound method PandasDelegate._add_delegate_acc...,29,5,2019,9,51,0,<bound method PandasDelegate._add_delegate_acc...,2019
3,MADRID,SEVILLA,AVE,69.40,Preferente,Promo,5,31,43,19,...,0,<bound method PandasDelegate._add_delegate_acc...,29,5,2019,10,32,0,<bound method PandasDelegate._add_delegate_acc...,2019
5,MADRID,SEVILLA,AVE,60.30,Turista,Promo,5,31,43,19,...,0,<bound method PandasDelegate._add_delegate_acc...,29,5,2019,11,38,0,<bound method PandasDelegate._add_delegate_acc...,2019


# Conclusion

We went through a simple workflow for preprocessing the dataset, then encoding and splitting it into training and test set. We then tested 3 different algorithms, i.e. a Linear Regression, an SVM, and Gradient Boosting. We can observe that gradient boosting, in this case, is not only faster, but significantly more accurate.